<a href="https://colab.research.google.com/github/sSteam-cube/Optics_lab_codes_and_data/blob/main/P343%3AMilikans_experiment/Milikans_oil_drop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dynamic Method


In [ ]:
import numpy as np
import pandas as pd
import statistics as stats

# Constants
C = 190.13
D = 9.04e-9
zeta = 4.06e-8
L = 1.0e-3  # distance between lines in m


dynamic_data =  [
    ([15.8, 16, 16.5, 19.5, 18.4], [8.1, 7.5, 8.1, 7.7, 7.6], 294),  # Droplet 1
    ([16.2, 17.0, 16.3, 16.8, 17.4], [4.3, 4.2, 4.2, 4.2, 4.1], 140),  # Droplet 2
    ([17.4, 15.6, 15.8, 15.7, 15.5], [2.1, 2.1, 2.2, 2.1, 2.4], 141),  # Droplet 3
    ([21.9, 20.2, 20.6, 19.6, 20.8], [7.0, 7.1, 7.1, 6.4, 7.1], 274),  # Droplet 4
    ([17.6, 18.0, 18.0, 16.2], [4.8, 4.8, 4.7, 5.2], 140),             # Droplet 5
    ([5.4, 5.4, 5.4, 5.7, 5.7], [4.4, 4.7, 4.3, 4.0, 4.5], 300)        # Droplet 6
]

# Compute means
data = [
    [stats.mean(pos), stats.mean(vel), val]
    for pos, vel, val in dynamic_data
]

results = []

for tf, tr, V in data:
    vf = L / tf                      # free-fall velocity
    xi = D * vf                      # ξ
    r = -zeta + np.sqrt(zeta**2 + xi)  # radius
    r3 = r**3
    T = 1 + (tf / tr)
    ne = (C * T * r3) / V            # charge
    results.append([xi, r, r3, T, ne])

# Make a table
df = pd.DataFrame(results, columns=["ξ (m²)", "r (m)", "r³ (m³)", "T", "ne (C)"])
print(df)

         ξ (m²)         r (m)       r³ (m³)         T        ne (C)
0  5.243619e-13  6.846657e-07  3.209487e-19  3.210256  6.663137e-19
1  5.400239e-13  6.953839e-07  3.362589e-19  4.985714  2.276795e-18
2  5.650000e-13  7.121605e-07  3.611883e-19  8.339450  4.061651e-18
3  4.384093e-13  6.227684e-07  2.415348e-19  3.971182  6.655790e-19
4  5.180516e-13  6.803022e-07  3.148514e-19  4.579487  1.958146e-18
5  1.637681e-12  1.239763e-06  1.905531e-18  2.260274  2.729647e-18


In [ ]:
# --- Continue from your previous code ---

# Extract ne column
ne_values = df["ne (C)"].to_numpy()

# Step 1: minimum value of ne
ne_min = ne_values.min()

# Step 2–4: build the table
table = []
for ne in ne_values:
    ratio = ne / ne_min
    n_eff = int(round(ratio))  # nearest integer
    e_val = ne / n_eff
    table.append([ne, ratio, n_eff, e_val])

# Step 5: average e
avg_e = np.mean([row[3] for row in table])

# Create DataFrame for clarity
df_dynamic = pd.DataFrame(
    table,
    columns=["ne (C)", "ne / lowest", "nearest integer (n_eff)", "ne / n_eff"]
)

print("Dynamic Method Data:\n", df_dynamic)
print("\nAverage elementary charge estimate =", avg_e)


Dynamic Method Data:
          ne (C)  ne / lowest  nearest integer (n_eff)    ne / n_eff
0  6.663137e-19     1.001104                        1  6.663137e-19
1  2.276795e-18     3.420773                        3  7.589315e-19
2  4.061651e-18     6.102432                        6  6.769418e-19
3  6.655790e-19     1.000000                        1  6.655790e-19
4  1.958146e-18     2.942019                        3  6.527153e-19
5  2.729647e-18     4.101162                        4  6.824118e-19

Average elementary charge estimate = 6.838154966323025e-19


# Balancing Method

In [ ]:
import numpy as np
import pandas as pd
import statistics as stats

# Constants (from manual)
C = 190.13
D = 9.04e-9
zeta = 4.06e-8
L = 1.0e-3  # distance between preset lines (m)

# Example input: ([free-fall times], Vb)
balancing_data = [
    ([15.8, 16, 16.5, 19.5, 18.4], 294),  # Droplet 1
    ([16.2, 17.0, 16.3, 16.8, 17.4], 140),  # Droplet 2
    ([17.4, 15.6, 15.8, 15.7, 15.5], 141),  # Droplet 3
    ([21.9, 20.2, 20.6, 19.6, 20.8], 274),  # Droplet 4
    ([17.6, 18.0, 18.0, 16.2], 140),        # Droplet 5
    ([5.4, 5.4, 5.4, 5.7, 5.7], 300)        # Droplet 6
]

results = []

for times, Vb in balancing_data:
    tf = stats.mean(times)       # mean free-fall time
    vf = L / tf                  # free-fall velocity
    xi = D * vf                  # ξ
    r = -zeta + np.sqrt(zeta**2 + xi)  # radius
    r3 = r**3
    ne = (C * r3) / Vb           # charge
    results.append([xi, r, r3, ne])

# Balancing Method Table
df_balancing = pd.DataFrame(
    results,
    columns=["ξ (m²)", "r (m)", "r³ (m³)", "ne (C)"],
    index=[f"Droplet {i+1}" for i in range(len(results))]
)

# --- Analysis and Treatment (Balancing Method) ---
ne_values = df_balancing["ne (C)"].to_numpy()
ne_min = ne_values.min()

analysis_table = []
for ne in ne_values:
    ratio = ne / ne_min
    n_eff = int(round(ratio))  # nearest integer
    e_val = ne / n_eff
    analysis_table.append([ne, ratio, n_eff, e_val])

df_analysis = pd.DataFrame(
    analysis_table,
    columns=["ne (C)", "ne / lowest", "nearest integer (n_eff)", "ne / n_eff"],
    index=[f"Droplet {i+1}" for i in range(len(analysis_table))]
)

avg_e = np.mean(df_analysis["ne / n_eff"])

# --- Output ---
pd.set_option("display.float_format", "{:.3e}".format)

print("Balancing Method Data:\n", df_balancing)
print("\nAnalysis & Treatment:\n", df_analysis)
print(f"\nFinal average e ≈ {avg_e:.3e} C")


Balancing Method Data:
              ξ (m²)     r (m)   r³ (m³)    ne (C)
Droplet 1 5.244e-13 6.847e-07 3.209e-19 2.076e-19
Droplet 2 5.400e-13 6.954e-07 3.363e-19 4.567e-19
Droplet 3 5.650e-13 7.122e-07 3.612e-19 4.870e-19
Droplet 4 4.384e-13 6.228e-07 2.415e-19 1.676e-19
Droplet 5 5.181e-13 6.803e-07 3.149e-19 4.276e-19
Droplet 6 1.638e-12 1.240e-06 1.906e-18 1.208e-18

Analysis & Treatment:
              ne (C)  ne / lowest  nearest integer (n_eff)  ne / n_eff
Droplet 1 2.076e-19    1.238e+00                        1   2.076e-19
Droplet 2 4.567e-19    2.725e+00                        3   1.522e-19
Droplet 3 4.870e-19    2.906e+00                        3   1.623e-19
Droplet 4 1.676e-19    1.000e+00                        1   1.676e-19
Droplet 5 4.276e-19    2.551e+00                        3   1.425e-19
Droplet 6 1.208e-18    7.206e+00                        7   1.725e-19

Final average e ≈ 1.675e-19 C
